In [2]:
import os
os.path.exists('models') or os.chdir("../")
print(os.getcwd())

/home/nitin/github/3D-ResNets-PyTorch


In [5]:
import torch
import torch.onnx as torch_onnx
from models import resnet
import onnx
from onnxsim import simplify

def export_model(
    model_path,
    onnx_path,
    n_classes,
    shape,
    batch_size=1,
    seq_length=16,
    model_depth=50,
    n_input_channels=3,
    shortcut_type="B",
    conv1_t_size=7,
    conv1_t_stride=1,
    no_max_pool=False,
    widen_factor=1.0):
    
    # A model class instance
    model = resnet.generate_model(
        model_depth=model_depth,
        n_classes=n_classes,
        n_input_channels=n_input_channels,
        shortcut_type=shortcut_type,
        conv1_t_size=conv1_t_size,
        conv1_t_stride=conv1_t_stride,
        no_max_pool=no_max_pool,
        widen_factor=widen_factor
     )

    # Load the weights from a file
    print(f"loading Pytorch Model: {model_path}")
    checkpoint = torch.load(model_path)

    # Load the weights now into a model net architecture
    if hasattr(model, 'module'):
        model.module.load_state_dict(checkpoint['state_dict'])
    else:
        model.load_state_dict(checkpoint['state_dict'])

    model.eval()    
    dummy_input = torch.randn(batch_size, 3, seq_length, shape[0], shape[1], requires_grad=True)
    out = model(dummy_input)
    torch_onnx.export(
        model, dummy_input, onnx_path,
        export_params=True, opset_version=10, do_constant_folding=True,
        input_names = ['input'], output_names = ['output'],
        dynamic_axes={'input' : {0 : 'batch_size'},'output' : {0 : 'batch_size'}}
    )

    # use onnxsimplify to reduce reduent model.
    input_shapes = {"input": list(dummy_input.shape)}
    onnx_model = onnx.load(onnx_path)
    model_simp, check = simplify(onnx_model, dynamic_input_shape=True, input_shapes=input_shapes)
    assert check, "Simplified ONNX model could not be validated"
    onnx.save(model_simp, onnx_path)
    print(f"generated simplified onnx model: {onnx_path}")


export_model(
    model_path="/home/nitin/github/kinetics/images/results/save_200.pth",
    onnx_path="kinetics_r50_122x122_test.onnx",
    shape=(112, 112),
    n_classes=5,
    seq_length=16
)

loading Pytorch Model: /home/nitin/github/kinetics/images/results/save_200.pth
generated simplified onnx model:kinetics_r50_122x122_test.onnx


In [6]:
import numpy as np
class Resnet3DONNX():
    import onnxruntime as ort

    def __init__(self, model_path, classes, shape=(112, 112), seq_length=16, mean=None, std=None):
        print(f'Using ONNX as inference backend')
        print(f'Using weight: {model_path}')
        self.model_path = model_path
        self.classes = classes
        self.shape = shape
        self.seq_length = seq_length
        self.mean = mean or [0.4345, 0.4051, 0.3775]
        self.std = std or [0.2768, 0.2713, 0.2737]
        self.ort_session = self.ort.InferenceSession(self.model_path)
        self.input_name = self.ort_session.get_inputs()[0].name
    
    def softmax(self, x, dim=0):
        if x.ndim == 1:
            x = x.reshape((1, -1))
        max_x = np.max(x, axis=1).reshape((-1, 1))
        exp_x = np.exp(x - max_x)
        return exp_x / np.sum(exp_x, axis=1).reshape((-1, 1))
    
    def preprocess(self, batch):
        batch_inputs = []
        for images in batch:
            inputs = []
            for img in images:
                img = cv2.resize(img, self.shape)
                img = ((img/255.0) - self.mean) / self.std
                inputs.append(img)
            batch_inputs.append(inputs[:self.seq_length])
        return np.array(batch_inputs).transpose(0,4,1,2,3)

    def __call__(self, inputs):
        return self.ort_session.run(None, {self.input_name: inputs})[0]
    
    def inference(self, batch):
        inputs = self.preprocess(batch)
        # batch,channel,frames,h,w
        outputs = self(inputs.astype(np.float32))
        outputs = self.softmax(outputs, dim=1)

        predictions = []
        for batch_output in outputs:
            p = sorted(list(zip(self.classes, batch_output)), key=lambda x: x[1], reverse=True)
            predictions.append(p)
        return predictions

onnx_model = Resnet3DONNX(
    "kinetics_r50_122x122_test.onnx",
    ['archery', 'bowling', 'marching', 'flying_kite', 'high_jump'],
    (112, 112)
)

Using ONNX as inference backend
Using weight: kinetics_r50_122x122_test.onnx


In [8]:
import os, glob
import cv2

def load_frames(path, limit=16):
    frames = []
    for path in glob.glob(path+"/*")[:limit]:
        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        frames.append(img)
    return frames

In [9]:
frames = load_frames("/home/nitin/github/kinetics/images/val/bowling/4JxH3S5JwMs_000003_000013")

In [10]:
onnx_model.inference([frames])[0]

[('bowling', 0.28196692),
 ('marching', 0.24497674),
 ('high_jump', 0.21826945),
 ('flying_kite', 0.13246316),
 ('archery', 0.12232374)]